# Timbr

[Timbr](https://docs.timbr.ai/doc/docs/integration/langchain-sdk/) integrates natural language inputs with Timbr's ontology-driven semantic layer. Leveraging Timbr's robust ontology capabilities, the SDK integrates with Timbr data models and leverages semantic relationships and annotations, enabling users to query data using business-friendly language.

This notebook provides a quick overview for getting started with Timbr tools and agents. For more information about Timbr visit [Timbr.ai](https://timbr.ai/) or the [Timbr Documentation](https://docs.timbr.ai/doc/docs/integration/langchain-sdk/)

## Overview

### Integration details

Timbr package for LangChain is [langchain-timbr](https://pypi.org/project/langchain-timbr), which provides seamless integration with Timbr's semantic layer for natural language to SQL conversion.

### Tool features

| Tool Name | Description |
| :--- | :--- |
| `IdentifyTimbrConceptChain` | Identify relevant concepts from user prompts |
| `GenerateTimbrSqlChain` | Generate SQL queries from natural language prompts |
| `ValidateTimbrSqlChain` | Validate SQL queries against Timbr knowledge graph schemas |
| `ExecuteTimbrQueryChain` | Execute SQL queries against Timbr knowledge graph databases |
| `GenerateAnswerChain` | Generate human-readable answers from query results |
| `TimbrSqlAgent` | End-to-end SQL agent for natural language queries |

### TimbrSqlAgent Parameters

The `TimbrSqlAgent` is a pre-built agent that combines all the above tools for end-to-end natural language to SQL processing.

For the complete list of parameters and detailed documentation, see: [TimbrSqlAgent Documentation](https://docs.timbr.ai/doc/docs/integration/langchain-sdk/#timbr-sql-agent)

| Parameter | Type | Required | Description |
| :--- | :--- | :--- | :--- |
| `llm` | BaseChatModel | Yes | Language model instance (ChatOpenAI, ChatAnthropic, etc.) |
| `url` | str | Yes | Timbr application URL |
| `token` | str | Yes | Timbr API token |
| `ontology` | str | Yes | Knowledge graph ontology name |
| `schema` | str | No | Database schema name |
| `concept` | str | No | Specific concept to focus on |
| `concepts_list` | List[str] | No | List of relevant concepts |
| `views_list` | List[str] | No | List of available views |
| `note` | str | No | Additional context or instructions |
| `retries` | int | No | Number of retry attempts (default: 3) |
| `should_validate_sql` | bool | No | Whether to validate generated SQL (default: True) |

## Setup

The integration lives in the `langchain-timbr` package.

In this example, we'll use OpenAI for the LLM provider.

In [1]:
%pip install --quiet -U langchain-timbr[openai]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### Credentials

You'll need Timbr credentials to use the tools. Get your API token from your Timbr application's API settings.

In [2]:
import getpass
import os

# Set up Timbr credentials
if not os.environ.get("TIMBR_URL"):
    os.environ["TIMBR_URL"] = input("Timbr URL:\n")

if not os.environ.get("TIMBR_TOKEN"):
    os.environ["TIMBR_TOKEN"] = getpass.getpass("Timbr API Token:\n")

if not os.environ.get("TIMBR_ONTOLOGY"):
    os.environ["TIMBR_ONTOLOGY"] = input("Timbr Ontology:\n")

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:\n")

## Instantiation

Instantiate Timbr tools and agents. First, let's set up the LLM and basic Timbr chains:

In [3]:
from langchain_timbr import (
    ExecuteTimbrQueryChain,
    GenerateAnswerChain,
    TimbrSqlAgent,
    LlmWrapper,
    LlmTypes
)

# Set up the LLM
# from langchain_openai import ChatOpenAI
# llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Alternative: Use Timbr's LlmWrapper for an easy LLM setup
llm = LlmWrapper(llm_type=LlmTypes.OpenAI, api_key=os.environ["OPENAI_API_KEY"], model="gpt-4o")

# Instantiate Timbr chains
execute_timbr_query_chain = ExecuteTimbrQueryChain(
    llm=llm,
    url=os.environ["TIMBR_URL"],
    token=os.environ["TIMBR_TOKEN"],
    ontology=os.environ["TIMBR_ONTOLOGY"]
)

generate_answer_chain = GenerateAnswerChain(
    llm=llm,
    url=os.environ["TIMBR_URL"],
    token=os.environ["TIMBR_TOKEN"]
)

## Invocation

### Execute SQL queries from natural language

You can use the individual chains to perform specific operations:

In [4]:
# Execute a natural language query
result = execute_timbr_query_chain.invoke({
    "prompt": "What are the total sales for last month?"
})

print("SQL Query:", result["sql"])
print("Results:", result["rows"])
print("Concept:", result["concept"])

# Generate a human-readable answer from the results
answer_result = generate_answer_chain.invoke({
    "prompt": "What are the total sales for last month?",
    "rows": result["rows"]
})

print("Human-readable answer:", answer_result["answer"])

C:\Users\barco\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
Token indices sequence length is longer than the specified maximum sequence length for this model (1242 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1242 > 1024). Running this sequence through the model will result in indexing errors


SQL Query: SELECT SUM(`measure.total_sales`) AS `total_sales_last_month`
FROM `vtimbr`.`order_cube`
WHERE `order_date` >= DATE_SUB(CURDATE(), INTERVAL '1' MONTH)
  AND `order_date` < CURDATE()
LIMIT 500
Results: [{'total_sales_last_month': None}]
Concept: order_cube
Human-readable answer: The query results indicate that the total sales for last month are not available or could not be determined, as the value is `None`. This could be due to missing data, an error in the query, or no sales being recorded for that period.
Human-readable answer: The query results indicate that the total sales for last month are not available or could not be determined, as the value is `None`. This could be due to missing data, an error in the query, or no sales being recorded for that period.


## Use within an agent

### Using TimbrSqlAgent

The `TimbrSqlAgent` provides an end-to-end solution that combines concept identification, SQL generation, validation, execution, and answer generation:

In [5]:
from langchain.agents import AgentExecutor

# Create a TimbrSqlAgent with all parameters
timbr_agent = TimbrSqlAgent(
    llm=llm,
    url=os.environ["TIMBR_URL"],
    token=os.environ["TIMBR_TOKEN"],
    ontology=os.environ["TIMBR_ONTOLOGY"],
    concepts_list=["Sales", "Orders"], # optional
    views_list=["sales_view"],         # optional
    note="Focus on monthly aggregations", # optional
    retries=3,                         # optional
    should_validate_sql=True           # optional
)

# Use the agent for end-to-end natural language to answer processing
agent_result = AgentExecutor.from_agent_and_tools(
    agent=timbr_agent,
    tools=[],  # No tools needed as we're directly using the chain
    verbose=True
).invoke("Show me the top 5 customers by total sales amount this year")

print("Final Answer:", agent_result["answer"])
print("Generated SQL:", agent_result["sql"])
print("Usage Metadata:", agent_result.get("usage_metadata", {}))



> Entering new AgentExecutor chain...
Error in TimbrSqlAgent.plan (sync): Error executing the chain: No relevant concepts found for the query.

> Finished chain.
Final Answer: None
Generated SQL: None
Usage Metadata: {}
Error in TimbrSqlAgent.plan (sync): Error executing the chain: No relevant concepts found for the query.

> Finished chain.
Final Answer: None
Generated SQL: None
Usage Metadata: {}


### Sequential Chains

You can combine multiple Timbr chains using LangChain's SequentialChain for custom workflows:

In [6]:
from langchain.chains import SequentialChain

# Create a sequential pipeline
pipeline = SequentialChain(
    chains=[execute_timbr_query_chain, generate_answer_chain],
    input_variables=["prompt"],
    output_variables=["answer", "sql", "rows"]
)

# Execute the pipeline
pipeline_result = pipeline.invoke({
    "prompt": "What are the average order values by customer segment?"
})

print("Pipeline Result:", pipeline_result)

Pipeline Result: {'prompt': 'What are the average order values by customer segment?', 'answer': 'The average order values by customer segment are as follows:\n\n- Consumer: 7,738,842.41\n- Home Office: 2,645,944.68\n- Corporate: 4,487,318.15', 'sql': 'SELECT     `customer_segment`, \n    AVG(`measure.total_revenue`) AS `average_order_value`\nFROM \n    `vtimbr`.`order_cube`\nGROUP BY \n    `customer_segment`\nLIMIT 500', 'rows': [{'customer_segment': 'Consumer', 'average_order_value': 7738842.413591726}, {'customer_segment': 'Home Office', 'average_order_value': 2645944.676290862}, {'customer_segment': 'Corporate', 'average_order_value': 4487318.148928828}]}


In [7]:
# Example: Accessing usage metadata from Timbr operations
result_with_metadata = execute_timbr_query_chain.invoke({
    "prompt": "How many orders were placed last quarter?"
})

# Extract usage metadata
usage_metadata = result_with_metadata.get("execute_timbr_usage_metadata", {})
determine_concept_usage = usage_metadata.get('determine_concept', {})
generate_sql_usage = usage_metadata.get('generate_sql', {})

print(determine_concept_usage)

print("Concept determination token estimate:", determine_concept_usage.get('approximate', 'N/A'))
print("Concept determination tokens:", determine_concept_usage.get('token_usage', {}).get('total_tokens', 'N/A'))

print("SQL generation token estimate:", generate_sql_usage.get('approximate', 'N/A'))
print("SQL generation tokens:", generate_sql_usage.get('token_usage', {}).get('total_tokens', 'N/A'))


{'approximate': 469, 'token_usage': {'completion_tokens': 1, 'prompt_tokens': 399, 'total_tokens': 400, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_80956533cb', 'id': 'chatcmpl-CBI66m5PKcgx29vHCXAC2TqI0U9tZ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}
Concept determination token estimate: 469
Concept determination tokens: 400
SQL generation token estimate: 3167
SQL generation tokens: 2659


## API reference

- [PyPI](https://pypi.org/project/langchain-timbr)
- [GitHub](https://github.com/WPSemantix/langchain-timbr)
- [LangChain Timbr Documentation](https://docs.timbr.ai/doc/docs/integration/langchain-sdk/)
- [LangGraph Timbr Documentation](https://docs.timbr.ai/doc/docs/integration/langgraph-sdk)
- [Timbr Official Website](https://timbr.ai/)